In [1]:
import pandas as pd
import numpy as np
import pathlib
import matplotlib.pyplot as plt
import seaborn as sns
import os
from tqdm import tqdm
from typing import List, Union, Optional, Callable
import pickle
from Bio import AlignIO, SeqIO
from ete3 import Tree, TreeNode
from gctree import CollapsedTree

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import umap
from ete3 import Tree, faces, TreeStyle, NodeStyle, TextFace, SequenceFace, COLOR_SCHEMES
from GCTree_preparation import *
import warnings
import math
warnings.filterwarnings("ignore")

path_to_storage = "/media/hieunguyen/HNSD01/storage/all_BSimons_datasets"
outdir = "/media/hieunguyen/HNSD_mini/outdir/sc_bulk_BCR_data_analysis_v0.1"

PROJECT = "220701_etc_biopsies"
path_to_main_output = f"{outdir}/tree_analysis/{PROJECT}"
path_to_01_output = os.path.join(path_to_main_output, "01_output")
os.system(f"mkdir -p {path_to_01_output}")

output_type = "mouse_based_output"

path_to_trees = os.path.join(path_to_storage, PROJECT, "GCtrees/v0.2", output_type)

all_tree_folder = [item for item in pathlib.Path(path_to_trees).glob("*") if 
                   os.path.isfile(f"{str(item)}/02_dnapars/gctree.out.inference.1.nk") == True]

all_nk_files = [item for item in pathlib.Path(path_to_trees).glob("*/*/*gctree.out.inference.1.nk")]  
print(f"Number of trees: {len(all_tree_folder)}")   

/home/hieunguyen/miniconda3/envs/ete3_py39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Number of trees: 7618


In [2]:
rerun = True
if (os.path.isfile(f"{path_to_01_output}/tree_summarydf.csv") == False) or (rerun == True):
    saveTreeobj = dict()
    # loop through all trees
    maindf = pd.DataFrame()
    # for treedir in tqdm(all_tree_folder[0:1]):
    for treedir in tqdm(all_tree_folder):
        cloneid = treedir.name
        treedir = str(treedir)

        mouseID = cloneid.split("_")[0]
        V_gene = cloneid.split("_")[1]
        J_gene = cloneid.split("_")[2]
        CDR3_len = cloneid.split("_")[3]

        nk_path = f"{treedir}/02_dnapars/gctree.out.inference.1.nk"
        ab_dict_path = os.path.join(treedir, "01_deduplicate", f"{cloneid}.abundance.csv")
        path_to_orig_fasta= f"{treedir}/01_deduplicate/{cloneid}.fasta"
        input_idmaps = f"{treedir}/01_deduplicate/{cloneid}.id_map_seq.csv"
        
        abund_df = pd.read_csv(ab_dict_path, index_col=0, names=['val'])
        ab_dict = abund_df.to_dict().get('val')
        tree_path = treedir
        tree = Tree(newick=nk_path, format=1)
        if ab_dict is not None:
            for node in tree.traverse():
                node.add_feature('abundance', ab_dict.get(node.name, 0))
        treeobj = GCtree(tree = tree, 
                        path = tree_path,
                        origin_fasta = path_to_orig_fasta,
                        idmap_seq = input_idmaps)
        saveTreeobj[cloneid] = treeobj
        seqdf_orig = treeobj.seqdf
        seqs = treeobj.seqs
        num_nodes = len(treeobj.nodes)
        num_leaves = len(treeobj.leaves)
        num_internal_nodes = len(treeobj.internal_nodes)
        num_passthrough_nodes = len(treeobj.passthrough_nodes)
        num_split_nodes = len(treeobj.split_nodes)
        num_observed_nodes = len(treeobj.observed_nodes)
        num_inferred_nodes = len(treeobj.inferred_nodes)
        count_single_node = treeobj.count_single_node
        count_mix_node = treeobj.count_mix_node
        tmpdf = pd.DataFrame(
            {
                "cloneid": cloneid,
                "mouseID": mouseID,
                "V_gene": V_gene,
                "J_gene": J_gene,
                "CDR3_len": CDR3_len,
                "num_nodes": num_nodes,
                "num_leaves": num_leaves,
                "num_internal_nodes": num_internal_nodes,
                "num_passthrough_nodes": num_passthrough_nodes,
                "num_split_nodes": num_split_nodes,
                "num_observed_nodes": num_observed_nodes,
                "num_inferred_nodes": num_inferred_nodes,
                "num_MID": len(seqdf_orig["MID"].unique()),
                "num_seq_fasta": len(seqs),
                "num_single_node": count_single_node,
                "num_mix_node": count_mix_node
            },
            index=[0],
        )
        maindf = pd.concat([maindf, tmpdf], axis = 0)
    maindf = maindf.sort_values(by = "num_MID", ascending = False)
    maindf.to_csv(f"{path_to_01_output}/tree_summarydf.csv", index = False)

100%|██████████| 7618/7618 [02:59<00:00, 42.40it/s]


In [3]:
treeobj = saveTreeobj["m30_IGHV2-9_IGHJ1-03_45_1.aln"]
color_path = "./hex_color.csv"
mid_color_pal = pd.read_csv(color_path, index_col = [0]).to_dict()["hex color"]

In [4]:
abund_pct = treeobj.abund_pct
def layout(n):
    size = max(1, 10 * math.sqrt(n.abundance))

    if n.abundance > 1:
        cols = [mid_color_pal[mid] for mid in abund_pct[n.name].keys()]
        values = [abund_pct[n.name][mid] for mid in abund_pct[n.name].keys()]
        F = faces.PieChartFace(values, colors=cols,
                                width=size * 2, height=size * 2)
        F.border.width = None
        # F.opacity = 0.6
        faces.add_face_to_node(F, n, 0, position="branch-right")
        ns = NodeStyle()
        ns["size"] = 0
        n.set_style(ns)

t = treeobj.tree

for n in t.traverse("postorder"):
    if n.abundance > 1:
        cols = [mid_color_pal[mid] for mid in abund_pct[n.name].keys()]
        values = [abund_pct[n.name][mid] for mid in abund_pct[n.name].keys()]

ts = TreeStyle()
ts.layout_fn = layout
ts.mode = "r"
ts.rotation = 90
ts.show_leaf_name = False
treeobj.tree.render("%%inline", tree_style=ts)

In [ ]:
maindf[maindf["num_mix_node"] != 0].sort_values(by = "num_mix_node", ascending = False)

,cloneid,mouseID,V_gene,J_gene,CDR3_len,num_nodes,num_leaves,num_internal_nodes,num_passthrough_nodes,num_split_nodes,num_observed_nodes,num_inferred_nodes,num_MID,num_seq_fasta,num_single_node,num_mix_node
0,m53_IGHV1-18-01_IGHJ2-02_54_1.aln,m53,IGHV1-18-01,IGHJ2-02,54,9,4,4,1,3,7,2,5,10,5,2
0,m53_IGHV1-53-01_IGHJ3-01_36_1.aln,m53,IGHV1-53-01,IGHJ3-01,36,20,9,10,4,6,14,6,5,19,13,1
0,m11_IGHV2-4-01_IGHJ1-03_48_1.aln,m11,IGHV2-4-01,IGHJ1-03,48,60,32,27,11,16,52,8,5,59,51,1
0,m53_IGHV1-64-01_IGHJ4-01_42_1.aln,m53,IGHV1-64-01,IGHJ4-01,42,52,28,23,9,14,45,7,5,58,44,1
0,m53_IGHV1-39-01_IGHJ1-03_63_1.aln,m53,IGHV1-39-01,IGHJ1-03,63,67,36,30,13,17,56,11,5,72,55,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,m37_IGHV1-54-01_IGHJ4-01_33_1.aln,m37,IGHV1-54-01,IGHJ4-01,33,3,1,1,1,0,2,1,2,4,1,1
0,m39_IGHV1-75-01_IGHJ2-01_51_5.aln,m39,IGHV1-75-01,IGHJ2-01,51,4,2,1,0,1,3,1,2,5,2,1
0,m12_IGHV3-6-01_IGHJ4-01_45_4.aln,m12,IGHV3-6-01,IGHJ4-01,45,4,2,1,0,1,2,2,2,4,1,1
0,m39_IGHV1-66-01_IGHJ3-01_39_1.aln,m39,IGHV1-66-01,IGHJ3-01,39,9,4,4,1,3,7,2,2,9,6,1
